In [1]:
NB_STATIONS = 839
BASE_NAME = "../all_stations/%s_features.csv"
DEST_NAME = "../all_stations_events/%s_events.csv"

## 2. Events
Extracted events from 2015-2019 from [London Events](http://www.londontown.com/London/London-Events-2019).</br>


**Assumption**: The district with the highest event density has the highest frequency.

In [2]:
import json
import pandas as pd
with open('eventsdata/events-2015.json', 'r') as f:
    data15 = json.load(f)
df15= pd.DataFrame(data15)
df1 =df15.drop(['locality','url'],axis=1)
# Data from 2016
with open('eventsdata/events-2016.json', 'r') as f:
    data16 = json.load(f)
df16= pd.DataFrame(data16)
df2 = df16.drop(['locality','url'],axis=1)
# Data from 2017
with open('eventsdata/events-2017.json', 'r') as f:
    data17 = json.load(f)
df17= pd.DataFrame(data17)
df3 = df17.drop(['locality','url'],axis=1)
# Data from 2018
with open('eventsdata/events-2018.json', 'r') as f:
    data18 = json.load(f)
df18= pd.DataFrame(data18)
df4 = df18.drop(['locality','url'],axis=1)
# Data from 2019
with open('eventsdata/events-2019.json', 'r') as f:
    data19 = json.load(f)
df19= pd.DataFrame(data19)
df5 = df19.drop(['locality','url'],axis=1)

In [3]:
# Merge data
frames = [df1, df2, df3,df4,df5]
final = pd.concat(frames)
#f = final.assign(Month='0')
final.title = final.title.map(lambda x: x.replace("\n", ""))
final.head()

,dates,postcode,street,title
0,05th - 06th June 2018,KT8 9AU,"East Molesey, Surrey",Lionel Richie
1,08th - 09th March 2019,SE1 8XX,Belvedere Road,Women of the World Festival
2,02nd - 14th March 2015,SW1E 5JA,12 Palace Street,Ruby Wax: Sane New World
3,03rd - 07th March 2015,TW9 1QJ,"The Green, Surrey",Harvey
4,03rd - 14th March 2015,NW1 3BF,"15-16 Triton Street, Regents Place",Citizen Puppet


In [4]:
from NominatimLibrary import Locator
locator = Locator()
geonames = pd.read_csv("geoname_postcodes_world.txt", sep="\t", names=["country","postcode","city","","","","","","","lat","lon","accuracy"])

dico = {}
def locate(dictionnary, locator, x):
    if dico.get(x) == None:
        try:
            dico[x] = locator.get_coordinates("" + x + ", England, GB")
        except Exception as e:
            # print(e)
            # If it was not found, we use only the three first letters:
            dpt = x[:3]
            geo_coords = geonames[(geonames["country"] == "GB") & (geonames["postcode"] == dpt)][["lat","lon"]]
            # Try in geonames :
            if len(geo_coords) > 0:
                dico[x] = (geo_coords.iloc[0].lat, geo_coords.iloc[0].lon)
            else:
                # Finally, try with Nominatim with the three first characters:
                try:
                    dico[x] = locator.get_coordinates("" + dpt + ", England, GB")
                except Exception as e:
                    print("Could not find :", e)
                    if x[:3] == "XX1":
                        print("Note: it is normal the postcode doesn't exist, it is 'secret cinema' at an unknown location.")
                    # Finally if nothing is found, we take a point very far away:
                    dico[x] = (0, 0)
    return dico[x]

final["coords"] = final["postcode"].map(lambda x: locate(dico, locator, x))
final.head()

/opt/anaconda/lib/python3.7/site-packages/pandas/io/parsers.py:702: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)
/opt/anaconda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,4,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Could not find : XX1, England, GB
Note: it is normal the postcode doesn't exist, it is 'secret cinema' at an unknown location.


,dates,postcode,street,title,coords
0,05th - 06th June 2018,KT8 9AU,"East Molesey, Surrey",Lionel Richie,"(51.3143224, -0.231344251983845)"
1,08th - 09th March 2019,SE1 8XX,Belvedere Road,Women of the World Festival,"(51.4976029, -0.0812871)"
2,02nd - 14th March 2015,SW1E 5JA,12 Palace Street,Ruby Wax: Sane New World,"(51.51301285, -0.0434090325310393)"
3,03rd - 07th March 2015,TW9 1QJ,"The Green, Surrey",Harvey,"(51.577252, -0.3714965)"
4,03rd - 14th March 2015,NW1 3BF,"15-16 Triton Street, Regents Place",Citizen Puppet,"(51.51301285, -0.0434090325310393)"


Fucking dataset, different formats for the dates:
- "day month year"
- "dow day month year"
- "day - day month year"
- "dow day - dow day month year"
- "day month - day month year"
- "day month to day month year"
- "Various Venues"
- "day and day month year"
- "day, day and day month year"
- "day, day, day and day month year"
- "day month, day month, day month and day month year"
- **Actually any combinations... !**

- Funny one : `15th May - 05th March 2016`

In [5]:
import re
from datetime import date, timedelta

# It would have been possible to make it more generic, but...
SINGLE_DATE = "^[A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) ([0-9]{4}).*"
TWO_DATES = "^[A-Za-z ]*([0-9]{1,2})[^ ]* and [A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) ([0-9]{4}).*"
THREE_DATES = "^[A-Za-z ]*([0-9]{1,2})[^ ]*, [A-Za-z ]*([0-9]{1,2})[^ ]* and [A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) ([0-9]{4}).*"
FOUR_DATES = "^[A-Za-z ]*([0-9]{1,2})[^ ]*, [A-Za-z ]*([0-9]{1,2})[^ ]*, [A-Za-z ]*([0-9]{1,2})[^ ]* and [A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) ([0-9]{4}).*"

TWO_MONTHS = "^[A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) and [A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) ([0-9]{4}).*"
THREE_MONTHS = "^[A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+), [A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) and [A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) ([0-9]{4}).*"
FOUR_MONTHS = "^[A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+), [A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+), [A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) and [A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) ([0-9]{4}).*"

SINGLE_DATE = "^[A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) ([0-9]{4}).*"
DOUBLE_DAYS = "^([0-9]{1,2})[^ ]* (-|to) [A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) ([0-9]{4}).*"
DOUBLE_MONTHS = "^([0-9]{1,2})[^ ]* ([A-Za-z]+) (-|to) [A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) ([0-9]{4}).*"
DOUBLE_YEARS = "^([0-9]{1,2})[^ ]* ([A-Za-z]+) ([0-9]{4}) (-|to) [A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) ([0-9]{4}).*"

MONTHS = [
    "January",
    "February",
    "March",
    "April",
    "May",
    "June",
    "July",
    "August",
    "September",
    "October",
    "November",
    "December",
]

def get_month_id(month):
    month_id = -1
    for (i,m) in enumerate(MONTHS):
        if m[:3] == month[:3]:
            month_id = i + 1
    return month_id

def make_date(match, year_id, month_id, day_id):
    year = int(match.group(year_id))
    month = get_month_id(match.group(month_id))
    day = int(match.group(day_id))
    return date(year,month,day)

def format_date(date_range):
    res = []
    start = None
    end = None

    if date_range == "Various Venues":
        return []
    
    match = re.match(DOUBLE_YEARS, date_range)
    if match != None:
        start = make_date(match,3,2,1)
        end = make_date(match,7,6,5)
    
    match = re.match(DOUBLE_MONTHS, date_range)
    if match != None:
        start = make_date(match,6,2,1)
        end = make_date(match,6,5,4)
    
    if start == None:   
        match = re.match(DOUBLE_DAYS, date_range)
        if match != None:
            start = make_date(match,5,4,1)
            end = make_date(match,5,4,3)
    
    if start != None:
        if start > end: # If only "day0 month0 - day1 month1 year" but month0 > month1, then 0 is a year before.
            start = start.replace(year=start.year - 1)
        while start <= end:
            res.append(start)
            start = start + timedelta(1)

    # If it is not a range:

    if len(res) == 0:
        match = re.match(SINGLE_DATE, date_range)
        if match != None:
            res.append(make_date(match,3,2,1))

    if len(res) == 0:
        match = re.match(TWO_DATES, date_range)
        if match != None:
            res.append(make_date(match,4,3,1))
            res.append(make_date(match,4,3,2))

    if len(res) == 0:
        match = re.match(THREE_DATES, date_range)
        if match != None:
            res.append(make_date(match,5,4,1))
            res.append(make_date(match,5,4,2))
            res.append(make_date(match,5,4,3))

    if len(res) == 0:
        match = re.match(FOUR_DATES, date_range)
        if match != None:
            res.append(make_date(match,6,5,1))
            res.append(make_date(match,6,5,2))
            res.append(make_date(match,6,5,3))
            res.append(make_date(match,6,5,4))
    
    if len(res) == 0:
        match = re.match(TWO_MONTHS, date_range)
        if match != None:
            res.append(make_date(match,5,2,1))
            res.append(make_date(match,5,4,3))

    if len(res) == 0:
        match = re.match(THREE_MONTHS, date_range)
        if match != None:
            res.append(make_date(match,7,2,1))
            res.append(make_date(match,7,4,3))
            res.append(make_date(match,7,6,5))

    if len(res) == 0:
        match = re.match(FOUR_MONTHS, date_range)
        if match != None:
            res.append(make_date(match,9,2,1))
            res.append(make_date(match,9,4,3))
            res.append(make_date(match,9,6,5))
            res.append(make_date(match,9,8,7))

    if len(res) == 0:
        print("Couldn't match: " + date_range)
    
    return res

final["dates_array"] = final["dates"].map(format_date)
final.head()
#format_date("15th May - 05th March 2016")

,dates,postcode,street,title,coords,dates_array
0,05th - 06th June 2018,KT8 9AU,"East Molesey, Surrey",Lionel Richie,"(51.3143224, -0.231344251983845)","[2018-06-05, 2018-06-06]"
1,08th - 09th March 2019,SE1 8XX,Belvedere Road,Women of the World Festival,"(51.4976029, -0.0812871)","[2019-03-08, 2019-03-09]"
2,02nd - 14th March 2015,SW1E 5JA,12 Palace Street,Ruby Wax: Sane New World,"(51.51301285, -0.0434090325310393)","[2015-03-02, 2015-03-03, 2015-03-04, 2015-03-0..."
3,03rd - 07th March 2015,TW9 1QJ,"The Green, Surrey",Harvey,"(51.577252, -0.3714965)","[2015-03-03, 2015-03-04, 2015-03-05, 2015-03-0..."
4,03rd - 14th March 2015,NW1 3BF,"15-16 Triton Street, Regents Place",Citizen Puppet,"(51.51301285, -0.0434090325310393)","[2015-03-03, 2015-03-04, 2015-03-05, 2015-03-0..."


In [6]:
events = final[["dates_array","title","coords"]]
events.to_csv("events_formatted.csv", sep=";")
events.head()

,dates_array,title,coords
0,"[2018-06-05, 2018-06-06]",Lionel Richie,"(51.3143224, -0.231344251983845)"
1,"[2019-03-08, 2019-03-09]",Women of the World Festival,"(51.4976029, -0.0812871)"
2,"[2015-03-02, 2015-03-03, 2015-03-04, 2015-03-0...",Ruby Wax: Sane New World,"(51.51301285, -0.0434090325310393)"
3,"[2015-03-03, 2015-03-04, 2015-03-05, 2015-03-0...",Harvey,"(51.577252, -0.3714965)"
4,"[2015-03-03, 2015-03-04, 2015-03-05, 2015-03-0...",Citizen Puppet,"(51.51301285, -0.0434090325310393)"


In [7]:
#events = pd.read_csv('eventsPrepared.csv', sep=";")
#events.head(75)

In [8]:
dates_events = {}

for index, row in events.iterrows():
    for d in row.dates_array:
        events_list = dates_events.get(d,[])
        events_list.append(row.coords)
        dates_events[d] = events_list

dates_events[date(2018,6,5)][:10]

[(51.3143224, -0.231344251983845),
 (51.51301285, -0.0434090325310393),
 (51.51301285, -0.0434090325310393),
 (51.51301285, -0.0434090325310393),
 (51.51301285, -0.0434090325310393),
 (51.5142, -0.1247),
 (51.51301285, -0.0434090325310393),
 (51.51301285, -0.0434090325310393),
 (51.43925785, -0.052486891876),
 (51.51301285, -0.0434090325310393)]

In [9]:
STATION_ID = 14
stations = pd.read_csv("../raw/rental_stations_saved.csv")
def get_station_coords(stations, station_id):
    coords = stations[(stations["id"] == STATION_ID)][["lat","lon"]].iloc[0]
    coords = (coords.lat, coords.lon)
    return coords

get_station_coords(stations, STATION_ID)

(51.52994371, -0.123616824)

In [16]:
# Function to add closest event distance and number of close events:

from datetime import datetime
from collections import deque

NO_EVENT_DISTANCE = 9999
CLOSE_LIMIT_KM = 3

def add_events(station_id, features):
    events_distances = []
    nb_close_events = []
    prev_station_id = -1
    
    # Pre-calculate distance:
    coords = get_station_coords(stations, station_id)
    closest_events = {}
    for row_date,row_events in dates_events.items():
        closest_event_distance = NO_EVENT_DISTANCE
        nb_close_evts = 0
        for e in row_events:
            dist = locator.distance_crow_coords(coords, e)
            if dist < closest_event_distance:
                closest_event_distance = dist
            if dist < CLOSE_LIMIT_KM:
                nb_close_evts += 1
                
        closest_events[row_date] = (closest_event_distance,nb_close_evts)

    # Append closest distance to the row:
    for index, row in features.iterrows():
        row_date = date(row["Year"],row["Month"],row["Day of Month"])
        closest,nb = closest_events.get(row_date, (NO_EVENT_DISTANCE,0))
        events_distances.append(closest)
        nb_close_events.append(nb)

    features["nearestEvent"] = events_distances
    events_distances = deque(events_distances)
    events_distances.rotate(-1)
    events_distances[0] = NO_EVENT_DISTANCE
    features["nearestEvent (Future)"] = events_distances
    
    features["nbCloseEvents"] = nb_close_events
    nb_close_events = deque(nb_close_events)
    nb_close_events.rotate(-1)
    nb_close_events[0] = 0
    features["nbCloseEvents (Future)"] = nb_close_events

In [19]:
# Running it for every stations:
def station(station_id):
    print(station_id, datetime.now())
    try:
        features = pd.read_csv(BASE_NAME % station_id)
        add_events(station_id, features)
        features.to_csv(DEST_NAME % station_id)
    except Exception as e:
        print(e)

station_ids = range(1, NB_STATIONS + 1)

from multiprocessing import Pool
pool = Pool()
pool.map(station, station_ids)

53 2019-05-31 15:56:28.140680
0 2019-05-31 15:56:28.140445
106 2019-05-31 15:56:28.140869
159 2019-05-31 15:56:28.141216
[Errno 2] File b'../all_stations/0_features.csv' does not exist: b'../all_stations/0_features.csv'
1 2019-05-31 15:56:28.537054
160 2019-05-31 15:57:18.346942
2 2019-05-31 15:57:18.347405
107 2019-05-31 15:57:18.346713
54 2019-05-31 15:57:18.345587
55 2019-05-31 15:57:54.176273
3 2019-05-31 15:57:55.835929
161 2019-05-31 15:57:55.974143
108 2019-05-31 15:57:56.443134


Process ForkPoolWorker-3:
Process ForkPoolWorker-1:
Process ForkPoolWorker-4:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anaconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/anaconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/anaconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/opt/anaconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/anaconda/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/opt/anaconda/lib/python3.7/multiprocessing/pool.py", line 

KeyboardInterrupt: 

KeyboardInterrupt
